## VERSION 6

this is the version that is working fully in terms of the agents and feedback.

In [ ]:
from openai import OpenAI
import time
from datetime import datetime
import json
import statistics
import re

# Setup OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

def generate_followup_question(learning_goal):
    prompt = f"""You are a motivational learning coach. A learner just said they want to learn: "{learning_goal}".

Generate one open-ended, thoughtful question that will help them expand deeply on their personal motivation, challenges, or relationship to this topic. Be emotionally engaging and not too abstract."""

    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1-0528-qwen3-8b:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=500
        )
        print("✅ DeepSeek response received.")
        print("🔧 FULL RESPONSE:", response)

        content = response.choices[0].message.content.strip() if response.choices else ""
        if not content:
            print("⚠️ Empty content. Using fallback.")
            content = "What does writing mean to you personally, and how do you want it to impact your future?"

        return content

    except Exception as e:
        print(f"⚠️ DeepSeek generation failed: {e}")
        return "What inspires your desire to write, and what do you hope to express through it?"

def agent_one():
    print("🎯 Welcome to the Adaptive Learning System (Prototype Agent 1)")
    learning_goal = input("🧠 What are you trying to learn today?\n> ")

    # Generate personalized follow-up
    followup_question = generate_followup_question(learning_goal)
    print(f"\n🔍 Reflect on this:\n{followup_question}\n")

    # Time response
    start_time = time.time()
    user_response = input("✍️ Your response:\n> ")
    end_time = time.time()

    time_taken = round(end_time - start_time, 2)
    timestamp = datetime.now().isoformat()

    print("\n🕒 Time to respond:", time_taken, "seconds")
    print("🗓️ Timestamp:", timestamp)

    return {
        "learning_goal": learning_goal,
        "followup_question": followup_question,
        "response": user_response,
        "time_taken": time_taken,
        "timestamp": timestamp
    }

def analyze_response_gemma(response_text, time_taken, session_num, goal, history=None):
    prompt = f"""
You are an expert language analyst. A learner wants to improve at: "{goal}".

Here is their reflective response:
"{response_text}"

Evaluate on a 0–1 scale:
1. Conceptual Vocabulary (depth, variety, originality)
2. Clarity of Expression
3. Emotional Tone Strength

Respond in JSON like this:
{{
  "conceptual_vocab_score": 0.0,
  "clarity_score": 0.0,
  "emotional_tone_score": 0.0
}}

Then write 1 sentence each explaining why the learner scored that way:
- Conceptual Vocabulary: ...
- Clarity: ...
- Emotional Tone: ...
"""
    try:
        result = client.chat.completions.create(
            model="google/gemma-3n-e2b-it:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4,
            max_tokens=300
        )
        raw_output = result.choices[0].message.content.strip()
        print("🔍 GEMMA RAW OUTPUT:", raw_output)

        # Extract JSON
        json_block = re.search(r"\{.*?\}", raw_output, re.DOTALL)
        if json_block:
            scores = json.loads(json_block.group(0))
        else:
            raise ValueError("Could not parse JSON block")

        # Extract explanation lines
        explanations = {
            "conceptual_vocab_feedback": "Not found",
            "clarity_feedback": "Not found",
            "emotional_tone_feedback": "Not found"
        }
        lines = raw_output.splitlines()
        for line in lines:
            if line.lower().startswith("- conceptual"):
                explanations["conceptual_vocab_feedback"] = line.split(":", 1)[1].strip()
            elif line.lower().startswith("- clarity"):
                explanations["clarity_feedback"] = line.split(":", 1)[1].strip()
            elif line.lower().startswith("- emotional"):
                explanations["emotional_tone_feedback"] = line.split(":", 1)[1].strip()

    except Exception as e:
        print("⚠️ Gemma scoring failed:", e)
        scores = {
            "conceptual_vocab_score": 0.5,
            "clarity_score": 0.5,
            "emotional_tone_score": 0.5
        }
        explanations = {
            "conceptual_vocab_feedback": "Default due to error.",
            "clarity_feedback": "Default due to error.",
            "emotional_tone_feedback": "Default due to error."
        }

    # Latency variability
    if history and len(history) > 1:
        previous_times = [s["time_taken"] for s in history]
        latency_std = round(statistics.stdev(previous_times), 2)
    else:
        latency_std = None

    return {
        **scores,
        **explanations,
        "time_to_completion": time_taken,
        "latency_variability": latency_std
    }

def generate_practice_pointers(goal, response, scores=None):
    trait_summary = ""
    if scores:
        trait_summary = f"""Their scores are:
- Conceptual Vocabulary: {scores.get('conceptual_vocab_score', 0.0)}
- Clarity: {scores.get('clarity_score', 0.0)}
- Emotional Tone: {scores.get('emotional_tone_score', 0.0)}"""

    prompt = f"""
You are a learning coach. A learner is trying to improve at: "{goal}".

Here is what they said:
"{response}"

Give 2–3 simple, specific, and practical next steps they can take to grow from here. 
Skip any scores or summaries. Just go straight to the point with motivational and clear tasks. 
Respond in bullet points like this:
- ...
- ...
"""


    try:
        result = client.chat.completions.create(
            model="qwen/qwen3-235b-a22b:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=500
        )
        suggestions = result.choices[0].message.content.strip()
        print("🧭 Agent 3 Practice Suggestions:\n", suggestions)
        return suggestions
    except Exception as e:
        print("⚠️ Qwen suggestion generation failed:", e)
        return "- Try writing a short journal about your thoughts.\n- Watch a simple video about your topic and reflect on what stands out."


# Main runner
if __name__ == "__main__":
    history = []

    result = agent_one()
    analysis = analyze_response_gemma(
        response_text=result["response"],
        time_taken=result["time_taken"],
        session_num=1,
        goal=result["learning_goal"],
        history=history
    )

    print("\n✅ DATA CAPTURED:")
    for k, v in result.items():
        print(f"{k}: {v}")

    print("\n📊 ANALYSIS SCORES:")
    for k in ["conceptual_vocab_score", "clarity_score", "emotional_tone_score"]:
        print(f"{k}: {analysis[k]}")

    print("\n🧠 FEEDBACK:")
    for k in ["conceptual_vocab_feedback", "clarity_feedback", "emotional_tone_feedback"]:
        label = k.replace("_feedback", "").replace("_", " ").capitalize()
        print(f"- {label}: {analysis[k]}")

    # ✅ Agent 3: Practice pointers
    practice_tips = generate_practice_pointers(
        goal=result["learning_goal"],
        response=result["response"],
        scores=analysis
    )

    print("\n🧭 NEXT STEPS FOR LEARNER:")
    print(practice_tips)

    # Save to history
    history.append(result)


In [ ]:
# ---------- this one works well for the first loop but produces too much outputs that we dont need and the second loops do not run well----------

# --------------------- installing important packagees. ---------------------------

from openai import OpenAI
import time
from datetime import datetime
import json
import statistics
import re

# Setup OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

def generate_followup_question(learning_goal):
    prompt = f"""You are a motivational learning coach. A learner just said they want to learn: "{learning_goal}".

Generate one open-ended, thoughtful question that will help them expand deeply on their personal motivation, challenges, or relationship to this topic. Be emotionally engaging and not too abstract."""

    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1-0528-qwen3-8b:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=500
        )
        print("✅ DeepSeek response received.")
        print("🔧 FULL RESPONSE:", response)

        content = response.choices[0].message.content.strip() if response.choices else ""
        if not content:
            print("⚠️ Empty content. Using fallback.")
            content = "What does writing mean to you personally, and how do you want it to impact your future?"

        return content

    except Exception as e:
        print(f"⚠️ DeepSeek generation failed: {e}")
        return "What inspires your desire to write, and what do you hope to express through it?"

def agent_one():
    print("🎯 Welcome to the Adaptive Learning System (Prototype Agent 1)")
    learning_goal = input("🧠 What are you trying to learn today?\n> ")

    # Generate personalized follow-up
    followup_question = generate_followup_question(learning_goal)
    print(f"\n🔍 Reflect on this:\n{followup_question}\n")

    # Time response
    start_time = time.time()
    user_response = input("✍️ Your response:\n> ")
    end_time = time.time()

    time_taken = round(end_time - start_time, 2)
    timestamp = datetime.now().isoformat()

    print("\n🕒 Time to respond:", time_taken, "seconds")
    print("🗓️ Timestamp:", timestamp)

    return {
        "learning_goal": learning_goal,
        "followup_question": followup_question,
        "response": user_response,
        "time_taken": time_taken,
        "timestamp": timestamp
    }

def analyze_response_gemma(response_text, time_taken, session_num, goal, history=None):
    prompt = f"""
You are an expert language analyst. A learner wants to improve at: "{goal}".

Here is their reflective response:
"{response_text}"

Evaluate on a 0–1 scale:
1. Conceptual Vocabulary (depth, variety, originality)
2. Clarity of Expression
3. Emotional Tone Strength

Respond in JSON like this:
{{
  "conceptual_vocab_score": 0.0,
  "clarity_score": 0.0,
  "emotional_tone_score": 0.0
}}

Then write 1 sentence each explaining why the learner scored that way:
- Conceptual Vocabulary: ...
- Clarity: ...
- Emotional Tone: ...
"""
    try:
        result = client.chat.completions.create(
            model="google/gemma-3n-e2b-it:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4,
            max_tokens=300
        )
        raw_output = result.choices[0].message.content.strip()
        print("🔍 GEMMA RAW OUTPUT:", raw_output)

        # Extract JSON
        json_block = re.search(r"\{.*?\}", raw_output, re.DOTALL)
        if json_block:
            scores = json.loads(json_block.group(0))
        else:
            raise ValueError("Could not parse JSON block")

        # Extract explanation lines
        explanations = {
            "conceptual_vocab_feedback": "Not found",
            "clarity_feedback": "Not found",
            "emotional_tone_feedback": "Not found"
        }
        lines = raw_output.splitlines()
        for line in lines:
            if line.lower().startswith("- conceptual"):
                explanations["conceptual_vocab_feedback"] = line.split(":", 1)[1].strip()
            elif line.lower().startswith("- clarity"):
                explanations["clarity_feedback"] = line.split(":", 1)[1].strip()
            elif line.lower().startswith("- emotional"):
                explanations["emotional_tone_feedback"] = line.split(":", 1)[1].strip()

    except Exception as e:
        print("⚠️ Gemma scoring failed:", e)
        scores = {
            "conceptual_vocab_score": 0.5,
            "clarity_score": 0.5,
            "emotional_tone_score": 0.5
        }
        explanations = {
            "conceptual_vocab_feedback": "Default due to error.",
            "clarity_feedback": "Default due to error.",
            "emotional_tone_feedback": "Default due to error."
        }

    # Latency variability
    if history and len(history) > 1:
        previous_times = [s["time_taken"] for s in history]
        latency_std = round(statistics.stdev(previous_times), 2)
    else:
        latency_std = None

    return {
        **scores,
        **explanations,
        "time_to_completion": time_taken,
        "latency_variability": latency_std
    }

def generate_practice_pointers(goal, response, scores=None):
    trait_summary = ""
    if scores:
        trait_summary = f"""Their scores are:
- Conceptual Vocabulary: {scores.get('conceptual_vocab_score', 0.0)}
- Clarity: {scores.get('clarity_score', 0.0)}
- Emotional Tone: {scores.get('emotional_tone_score', 0.0)}"""

    prompt = f"""
    You are an adaptive learning coach helping a learner improve at: "{goal}".

    Here is what they said:
    "{response}"

    Their current skill indicators (0–1 scale) are:
    - Conceptual Vocabulary: {scores.get('conceptual_vocab_score', 0.0)}
    - Clarity: {scores.get('clarity_score', 0.0)}
    - Emotional Tone: {scores.get('emotional_tone_score', 0.0)}

    Suggest 2–3 specific, doable next steps they can take to improve based on their current level. Be brief, direct, and realistic.

    💡 Respond ONLY with bullet points. Do not explain, repeat input, or include internal thoughts.
    """


    try:
        result = client.chat.completions.create(
            model="qwen/qwen3-235b-a22b:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=500
        )
        suggestions = result.choices[0].message.content.strip()
        print("🧭 Agent 3 Practice Suggestions:\n", suggestions)
        return suggestions
    except Exception as e:
        print("⚠️ Qwen suggestion generation failed:", e)
        return "- Try writing a short journal about your thoughts.\n- Watch a simple video about your topic and reflect on what stands out."


# ------------------- app starts running from here.-------------------------------

def run_learning_loop():
    history = []
    session_num = 1

    print("🎯 Welcome to the Adaptive Learning System")
    learning_goal = input("🧠 What are you trying to learn today?\n> ")

    while True:
        print(f"\n--- Session {session_num} ---")

        if session_num == 1:
            # Ask follow-up only once
            followup_question = generate_followup_question(learning_goal)
            print(f"\n🔍 Reflect on this:\n{followup_question}\n")
        else:
            # Use a consistent follow-up prompt or evolve it (optional)
            followup_question = f"Reflect again on your progress learning: {learning_goal}. Share what's clearer or still hard."

        # Capture user input
        start_time = time.time()
        user_response = input("✍️ Your response:\n> ")
        end_time = time.time()

        time_taken = round(end_time - start_time, 2)
        timestamp = datetime.now().isoformat()

        result = {
            "learning_goal": learning_goal,
            "followup_question": followup_question,
            "response": user_response,
            "time_taken": time_taken,
            "timestamp": timestamp
        }

        analysis = analyze_response_gemma(
            response_text=result["response"],
            time_taken=result["time_taken"],
            session_num=session_num,
            goal=learning_goal,
            history=history  # now used for progression
        )

        suggestions = generate_practice_pointers(
            goal=learning_goal,
            response=result["response"],
            scores=analysis
        )

        print("\n✅ DATA CAPTURED:")
        for k, v in result.items():
            print(f"{k}: {v}")

        print("\n📊 ANALYSIS SCORES:")
        for k in ["conceptual_vocab_score", "clarity_score", "emotional_tone_score"]:
            print(f"{k}: {analysis[k]}")

        print("\n🧠 FEEDBACK:")
        for k in ["conceptual_vocab_feedback", "clarity_feedback", "emotional_tone_feedback"]:
            label = k.replace("_feedback", "").replace("_", " ").capitalize()
            print(f"- {label}: {analysis[k]}")

        print("\n🧭 NEXT STEPS FOR LEARNER:")
        print(suggestions)

        # Store everything for memory
        history.append({
            **result,
            **analysis,
            "practice_suggestions": suggestions,
            "session_num": session_num
        })

        session_num += 1
        cont = input("\n🔁 Would you like to continue with another reflection session? (y/n): ").strip().lower()
        if cont != 'y':
            break


if __name__ == "__main__":
    run_learning_loop()






🎯 Welcome to the Adaptive Learning System


In [ ]:
# ----- this version works really well and gets to the start of the second loop but does not complete it so if we can get this part sorted we can move on with deployment------------------

# --------------------- installing important packagees. ---------------------------

from openai import OpenAI
import time
from datetime import datetime
import json
import statistics
import re

# Setup OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

def generate_followup_question(learning_goal):
    prompt = f"""You are a motivational learning coach. A learner just said they want to learn: "{learning_goal}".

Generate one open-ended, thoughtful question that will help them expand deeply on their personal motivation, challenges, or relationship to this topic. Be emotionally engaging and not too abstract."""

    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1-0528-qwen3-8b:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=500
        )
        print("✅ DeepSeek response received.")
        print("🔧 FULL RESPONSE:", response)

        content = response.choices[0].message.content.strip() if response.choices else ""
        if not content:
            print("⚠️ Empty content. Using fallback.")
            content = "What does writing mean to you personally, and how do you want it to impact your future?"

        return content

    except Exception as e:
        print(f"⚠️ DeepSeek generation failed: {e}")
        return "What inspires your desire to write, and what do you hope to express through it?"

def agent_one():
    print("🎯 Welcome to the Adaptive Learning System (Prototype Agent 1)")
    learning_goal = input("🧠 What are you trying to learn today?\n> ")

    # Generate personalized follow-up
    followup_question = generate_followup_question(learning_goal)
    print(f"\n🔍 Reflect on this:\n{followup_question}\n")

    # Time response
    start_time = time.time()
    user_response = input("✍️ Your response:\n> ")
    end_time = time.time()

    time_taken = round(end_time - start_time, 2)
    timestamp = datetime.now().isoformat()

    print("\n🕒 Time to respond:", time_taken, "seconds")
    print("🗓️ Timestamp:", timestamp)

    return {
        "learning_goal": learning_goal,
        "followup_question": followup_question,
        "response": user_response,
        "time_taken": time_taken,
        "timestamp": timestamp
    }

def analyze_response_gemma(response_text, time_taken, session_num, goal, history=None):
    prompt = f"""
You are an expert language analyst. A learner wants to improve at: "{goal}".

Here is their reflective response:
"{response_text}"

Evaluate on a 0–1 scale:
1. Conceptual Vocabulary (depth, variety, originality)
2. Clarity of Expression
3. Emotional Tone Strength

Respond in JSON like this:
{{
  "conceptual_vocab_score": 0.0,
  "clarity_score": 0.0,
  "emotional_tone_score": 0.0
}}

Then write 1 sentence each explaining why the learner scored that way:
- Conceptual Vocabulary: ...
- Clarity: ...
- Emotional Tone: ...
"""
    try:
        result = client.chat.completions.create(
            model="google/gemma-3n-e2b-it:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4,
            max_tokens=300
        )
        raw_output = result.choices[0].message.content.strip()
        print("🔍 GEMMA RAW OUTPUT:", raw_output)

        # Extract JSON
        json_block = re.search(r"\{.*?\}", raw_output, re.DOTALL)
        if json_block:
            scores = json.loads(json_block.group(0))
        else:
            raise ValueError("Could not parse JSON block")

        # Extract explanation lines
        explanations = {
            "conceptual_vocab_feedback": "Not found",
            "clarity_feedback": "Not found",
            "emotional_tone_feedback": "Not found"
        }
        lines = raw_output.splitlines()
        for line in lines:
            if line.lower().startswith("- conceptual"):
                explanations["conceptual_vocab_feedback"] = line.split(":", 1)[1].strip()
            elif line.lower().startswith("- clarity"):
                explanations["clarity_feedback"] = line.split(":", 1)[1].strip()
            elif line.lower().startswith("- emotional"):
                explanations["emotional_tone_feedback"] = line.split(":", 1)[1].strip()

    except Exception as e:
        print("⚠️ Gemma scoring failed:", e)
        scores = {
            "conceptual_vocab_score": 0.5,
            "clarity_score": 0.5,
            "emotional_tone_score": 0.5
        }
        explanations = {
            "conceptual_vocab_feedback": "Default due to error.",
            "clarity_feedback": "Default due to error.",
            "emotional_tone_feedback": "Default due to error."
        }

    # Latency variability
    if history and len(history) > 1:
        previous_times = [s["time_taken"] for s in history]
        latency_std = round(statistics.stdev(previous_times), 2)
    else:
        latency_std = None

    return {
        **scores,
        **explanations,
        "time_to_completion": time_taken,
        "latency_variability": latency_std
    }

def generate_practice_pointers(goal, response, scores=None):
    trait_summary = ""
    if scores:
        trait_summary = f"""Their scores are:
- Conceptual Vocabulary: {scores.get('conceptual_vocab_score', 0.0)}
- Clarity: {scores.get('clarity_score', 0.0)}
- Emotional Tone: {scores.get('emotional_tone_score', 0.0)}"""

    prompt = f"""
    You are an adaptive learning coach helping a learner improve at: "{goal}".

    Here is what they said:
    "{response}"

    Their current skill indicators (0–1 scale) are:
    - Conceptual Vocabulary: {scores.get('conceptual_vocab_score', 0.0)}
    - Clarity: {scores.get('clarity_score', 0.0)}
    - Emotional Tone: {scores.get('emotional_tone_score', 0.0)}

    Suggest 2–3 specific, doable next steps they can take to improve based on their current level. Be brief, direct, and realistic.

    💡 Respond ONLY with bullet points. Do not explain, repeat input, or include internal thoughts.
    """


    try:
        result = client.chat.completions.create(
            model="qwen/qwen3-235b-a22b:free",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=500
        )
        suggestions = result.choices[0].message.content.strip()
        print("🧭 Agent 3 Practice Suggestions:\n", suggestions)
        return suggestions
    except Exception as e:
        print("⚠️ Qwen suggestion generation failed:", e)
        return "- Try writing a short journal about your thoughts.\n- Watch a simple video about your topic and reflect on what stands out."


# ------------------- app starts running from here.-------------------------------

def run_learning_loop():
    history = []
    session_num = 1
    learning_goal = input("🎯 What are you trying to learn today?\n> ")

    followup_question = generate_followup_question(learning_goal)

    while True:
        print(f"\n--- Session {session_num} ---")

        if session_num == 1:
            print(f"\n🔍 Reflect on this:\n{followup_question}\n")
            start_time = time.time()
            user_response = input("✍️ Your response:\n> ")
            end_time = time.time()
        else:
            print(f"\n🔁 Reflect: What progress have you made toward '{learning_goal}' since last session?")
            progress_update = input("📈 Your progress:\n> ")

            # generate new reflective question based on update
            prompt = f"""The learner said their goal is: "{learning_goal}". 
They just shared this progress update: "{progress_update}". 

Now generate a new emotionally insightful follow-up reflection prompt that will help them think deeply about how to improve or stay motivated."""
            followup_question = client.chat.completions.create(
                model="deepseek/deepseek-r1-0528-qwen3-8b:free",
                messages=[{"role": "user", "content": prompt}]
            ).choices[0].message.content.strip()

            print(f"\n🔍 Reflect on this:\n{followup_question}\n")
            start_time = time.time()
            user_response = input("✍️ Your response:\n> ")
            end_time = time.time()

        time_taken = round(end_time - start_time, 2)
        timestamp = datetime.now().isoformat()

        result = {
            "session_num": session_num,
            "learning_goal": learning_goal,
            "followup_question": followup_question,
            "response": user_response,
            "time_taken": time_taken,
            "timestamp": timestamp
        }

        analysis = analyze_response_gemma(
            response_text=user_response,
            time_taken=time_taken,
            session_num=session_num,
            goal=learning_goal,
            history=history
        )

        suggestions = generate_practice_pointers(
            goal=learning_goal,
            response=user_response,
            scores=analysis
        )

        # Display final cleaned output
        print("\n✅ SESSION SUMMARY:")
        print(f"📆 Timestamp: {timestamp}")
        print(f"🧠 Goal: {learning_goal}")
        print(f"🗣️ Reflection: {user_response}")
        print("\n📊 SCORES:")
        for key in ["conceptual_vocab_score", "clarity_score", "emotional_tone_score"]:
            print(f"• {key.replace('_', ' ').title()}: {analysis[key]}")

        print("\n💬 FEEDBACK:")
        for key in ["conceptual_vocab_feedback", "clarity_feedback", "emotional_tone_feedback"]:
            print(f"• {analysis[key]}")

        print("\n🎯 NEXT STEPS:")
        print(suggestions)

        # Store session data
        history.append({
            **result,
            **analysis,
            "practice_suggestions": suggestions,
        })

        session_num += 1
        cont = input("\n🔁 Do another session? (y/n): ").strip().lower()
        if cont != "y":
            break



if __name__ == "__main__":
    run_learning_loop()






✅ DeepSeek response received.
🔧 FULL RESPONSE: ChatCompletion(id='gen-1752482150-x0ytngkaqm33c2G9qRoO', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Okay, I hear you! Learning something new is always exciting. To understand what truly drives you, let's explore it a bit deeper.\n\n**Question:**\n\nWhat specific moments, feelings, or experiences related to [**the topic you want to learn**] already excite you or have sparked your interest, even if just a little?", refusal=None, role='assistant', function_call=None, tool_calls=None, reasoning="Okay, the user wants me to act as a motivational learning coach. They've given me an empty quote where a learner expressed interest in learning something, and I need to generate one thoughtful, open-ended question that will help them explore their personal motivation, challenges, or relationship to the topic.\n\nFirst, I need to understand what the learner might be thinking. Their empty quote su